<a href="https://colab.research.google.com/github/mateokilla/twp/blob/main/Dividend_Yield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd

def get_dividend_and_fundamentals(tickers):
    results = {}

    for ticker in tickers:
        try:
            tk = yf.Ticker(ticker)
            info = tk.info

            # Osztalékhozam
            dy = info.get('dividendYield', None)

            # Következő várható osztalék dátum becslés
            dividends = tk.dividends
            next_div = None
            if not dividends.empty:
                div_dates = dividends.index.sort_values()
                last_div = div_dates[-1]
                if len(div_dates) > 1:
                    freq_days = (div_dates[-1] - div_dates[-2]).days
                    next_div_est = last_div + pd.Timedelta(days=freq_days)
                    next_div = next_div_est.strftime('%Y-%m-%d')

            # Fundamentális mutatók
            pe = info.get("trailingPE")
            forward_pe = info.get("forwardPE")
            roe = info.get("returnOnEquity")
            payout_ratio = info.get("payoutRatio")
            debt_equity = info.get("debtToEquity")
            target_price = info.get("targetMeanPrice")

            # Ár adatok
            current_price = info.get("currentPrice")
            low_52w = info.get("fiftyTwoWeekLow")
            high_52w = info.get("fiftyTwoWeekHigh")

            # Célár differencia
            diff_dollar = None
            if target_price and current_price:
                diff_dollar = target_price - current_price

            # g számítás (Damodaran fenntartható növekedési ráta)
            g_value = None
            if roe is not None and payout_ratio is not None:
                g_value = roe * (1 - payout_ratio)

            # Total Return (DY + g)
            total_return = None
            if dy is not None and g_value is not None:
                total_return = dy + (g_value * 100)

            results[ticker] = {
                'DY (%)': dy if dy is not None else None,
                'g (%)': g_value * 100 if g_value is not None else None,
                'Total Return (%)': total_return if total_return is not None else None,
                'Ex-Div': next_div,
                'P/E': pe,
                'Forward P/E': forward_pe,
                'ROE (%)': roe * 100 if roe is not None else None,
                'Payout Ratio (%)': payout_ratio * 100 if payout_ratio is not None else None,
                'Debt/Equity': debt_equity,
                'Price': current_price,
                '52W Low': low_52w,
                '52W High': high_52w,
                'Target Price': target_price,
                'Diff ($)': diff_dollar
            }
        except Exception as e:
            results[ticker] = {k: None for k in [
                'DY (%)', 'g (%)', 'Total Return (%)', 'Ex-Div',
                'P/E', 'Forward P/E', 'ROE (%)', 'Payout Ratio (%)',
                'Debt/Equity', 'Price', '52W Low', '52W High',
                'Target Price', 'Diff ($)'
            ]}
            print(f"Hiba a {ticker} lekérdezésekor: {e}")
    return results

if __name__ == '__main__':
    ticker_list = ["GAIN", "CSWC", "STAG", "O", "MMM", "SO", "PFE", "VZ", "T", "CVX"]
    data = get_dividend_and_fundamentals(ticker_list)

    header = f"{'Ticker':<6} | {'DY (%)':>7} | {'g (%)':>7} | {'TotalR (%)':>10} | {'Ex-Div':>12} | {'P/E':>6} | {'Fwd P/E':>7} | {'ROE (%)':>8} | {'Payout (%)':>10} | {'D/E':>6} | {'Price':>8} | {'52W Low':>8} | {'52W High':>8} | {'Target':>8} | {'Diff $':>8}"
    print(header)
    print("-" * len(header))

    for tk, vals in data.items():
        dy_str = f"{vals['DY (%)']:.2f}%" if vals['DY (%)'] is not None else "N/A"
        g_str = f"{vals['g (%)']:.2f}%" if vals['g (%)'] is not None else "N/A"
        total_str = f"{vals['Total Return (%)']:.2f}%" if vals['Total Return (%)'] is not None else "N/A"
        ex_div_str = vals['Ex-Div'] if vals['Ex-Div'] else "N/A"
        pe_str = f"{vals['P/E']:.2f}" if vals['P/E'] is not None else "N/A"
        fpe_str = f"{vals['Forward P/E']:.2f}" if vals['Forward P/E'] is not None else "N/A"
        roe_str = f"{vals['ROE (%)']:.2f}%" if vals['ROE (%)'] is not None else "N/A"
        payout_str = f"{vals['Payout Ratio (%)']:.2f}%" if vals['Payout Ratio (%)'] is not None else "N/A"
        de_str = f"{vals['Debt/Equity']:.2f}" if vals['Debt/Equity'] is not None else "N/A"
        price_str = f"${vals['Price']:.2f}" if vals['Price'] is not None else "N/A"
        low_str = f"${vals['52W Low']:.2f}" if vals['52W Low'] is not None else "N/A"
        high_str = f"${vals['52W High']:.2f}" if vals['52W High'] is not None else "N/A"
        target_str = f"${vals['Target Price']:.2f}" if vals['Target Price'] is not None else "N/A"
        diff_str = f"${vals['Diff ($)']:.2f}" if vals['Diff ($)'] is not None else "N/A"

        print(f"{tk:<6} | {dy_str:>7} | {g_str:>7} | {total_str:>10} | {ex_div_str:>12} | {pe_str:>6} | {fpe_str:>7} | {roe_str:>8} | {payout_str:>10} | {de_str:>6} | {price_str:>8} | {low_str:>8} | {high_str:>8} | {target_str:>8} | {diff_str:>8}")


Ticker |  DY (%) |   g (%) | TotalR (%) |       Ex-Div |    P/E | Fwd P/E |  ROE (%) | Payout (%) |    D/E |    Price |  52W Low | 52W High |   Target |   Diff $
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
GAIN   |   6.81% |   6.07% |     12.88% |   2025-08-21 |   7.91 |   15.48 |   13.17% |     53.93% |  91.31 |   $14.09 |   $11.42 |   $15.34 |   $13.92 |   $-0.17
CSWC   |  11.08% |  -6.27% |      4.81% |   2025-08-16 |  15.71 |    9.46 |    8.61% |    172.79% | 108.81 |   $23.09 |   $17.46 |   $26.00 |   $23.00 |   $-0.09
STAG   |   4.17% |  -0.82% |      3.35% |   2025-07-31 |  26.62 |   40.24 |    7.15% |    111.47% |  86.58 |   $35.41 |   $28.61 |   $41.63 |   $38.82 |    $3.41
O      |   5.63% |  -4.70% |      0.93% |   2025-07-30 |  52.05 |   36.01 |    2.51% |    287.59% |  69.96 |   $57.26 |   $50.71 |   $64.88 |   $62.03 |    $4.77
MMM    |   1.92% |  57.12% |